Задание:
1. Создайте виртуальную машину (можно без GPU)
2. Напишите скрипт, использующий "тяжелую" модель (YOLO, MASK-RCNN или создайте произвольную большую архитектуру по любому из занятий). Цель: долгое время predict'а модели. Либо отправлять на сервер батч изображений
3. Проведите тестирования обращения к Вашей моделе, развернутой на WSGI и на ASGI серверах
4. Ответом на ДЗ может быть таблица с результатами тестирования

In [ ]:
# Используемые библиотеки
%%capture
!pip install fastapi python-multipart ultralytics uvicorn gunicorn

In [ ]:
import gdown
# Модель и тестовые данные
gdown.download('https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov9e.pt', None, quiet=True)
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/knowledge/test_images.zip', None, quiet=True)
!unzip -qo test_images.zip

# '2. Напишите скрипт, использующий "тяжелую" модель'

In [ ]:
%%file flaskapp.py
import io
import time
from flask import Flask, request, jsonify
from ultralytics import YOLO
from PIL import Image

app = Flask(__name__)

# Загрузка модели
model = YOLO('yolov9e.pt')

@app.route('/predict', methods=['POST'])
def predict():
    if 'images' in request.files:
        images = request.files.getlist('images')
        # time.sleep(1)
        objects_info_all = []
        processing_times = []

        for image in images:
            image = Image.open(image)
            if image.mode != "RGB":
                image = image.convert("RGB")

            start_time = time.time()
            results = model(image)
            end_time = time.time()

            processing_time = end_time - start_time
            processing_times.append(processing_time)

            detections = results[0].boxes

            objects_info = []
            for det in detections:
                cls_id = int(det.cls)
                cls_name = model.names[cls_id]
                score = float(det.conf) * 100

                objects_info.append({
                    'class_id': cls_id,
                    'class_name': cls_name,
                    'score': score
                })

            objects_info_all.append({
                'objects': objects_info,
                'processing_time': processing_time
            })

        total_time = sum(processing_times)
        average_time = total_time / len(processing_times)

        return jsonify({
            'results': objects_info_all,
            'total_processing_time': total_time,
            'average_processing_time': average_time
        })

    return jsonify({'ошибка': 'Изображения не предоставлены'}), 400

if __name__ == '__main__':
    app.run(debug=True)


Writing flaskapp.py


In [ ]:
%%file fastapiapp.py
import io
import time
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import List
from PIL import Image
from ultralytics import YOLO

app = FastAPI()

# Загрузка модели
model = YOLO('yolov9e.pt')  # замените на реальный путь к вашей модели

class PredictionResult(BaseModel):
    class_id: int
    class_name: str
    score: float

class ImageResult(BaseModel):
    objects: List[PredictionResult]
    processing_time: float

class BatchResult(BaseModel):
    results: List[ImageResult]
    total_processing_time: float
    average_processing_time: float

@app.post("/predict", response_model=BatchResult)
async def predict(images: List[UploadFile] = File(...)):
    objects_info_all = []
    processing_times = []

    for image in images:
        image_data = await image.read()
        image = Image.open(io.BytesIO(image_data))
        if image.mode != "RGB":
            image = image.convert("RGB")

        start_time = time.time()
        results = model(image)
        end_time = time.time()

        processing_time = end_time - start_time
        processing_times.append(processing_time)

        detections = results[0].boxes

        objects_info = []
        for det in detections:
            cls_id = int(det.cls)
            cls_name = model.names[cls_id]
            score = float(det.conf) * 100  # Уверенность в процентах

            objects_info.append(PredictionResult(
                class_id=cls_id,
                class_name=cls_name,
                score=score
            ))

        objects_info_all.append(ImageResult(
            objects=objects_info,
            processing_time=processing_time
        ))

    total_time = sum(processing_times)
    average_time = total_time / len(processing_times)

    return BatchResult(
        results=objects_info_all,
        total_processing_time=total_time,
        average_processing_time=average_time
    )

# if __name__ == '__main__':
#     import uvicorn
#     uvicorn.run(app, host="0.0.0.0", port=8090)

if __name__ == '__main__':
    app.run(debug=True)


Overwriting fastapiapp.py


# '1. Создайте виртуальную машину' и '3. Проведите тестирования обращения к Вашей моделе, развернутой на WSGI и на ASGI серверах'

In [ ]:
import os
import glob
import requests
import time

# Убеждаемся, что нет запущенных экземпляров gunicorn
os.system('pkill gunicorn')

!nohup gunicorn --bind 0.0.0.0:8000 flaskapp:app &

# Задержка для запуска сервера
time.sleep(5)

url = 'http://0.0.0.0:8000/predict'
image_folder = '/content/test_images'

# Получаем список всех изображений в папке
image_paths = glob.glob(os.path.join(image_folder, '*'))

# Переменная для сохранения результатов
results_summary = []

# Отправляем изображения на сервер по одному
for image_path in image_paths:
    with open(image_path, 'rb') as f:
        files = {'images': f}
        response = requests.post(url, files=files)

    if response.status_code == 200:
        data = response.json()
        results_summary.append(data)
        print(f'Results for {image_path}:')
        for obj in data['results'][0]['objects']:
            print(f"Класс: {obj['class_name']}, Вероятность: {obj['score']:.2f}%")
        print(f"Время обработки этого изображения: {data['results'][0]['processing_time']:.2f} секунд\n")
    else:
        print(f"Обработка ошибок {image_path}: {response.text}")

# Выводим общую информацию по скорости обработки всех изображений
total_time_flask = sum([res['total_processing_time'] for res in results_summary])
average_time = total_time_flask / len(results_summary)

print(f"Время обработки всех изображений: {total_time_flask:.2f} сек")
print(f"Среднее время обработки одного изображения: {average_time:.2f} сек")


nohup: appending output to 'nohup.out'
Results for /content/test_images/10.jpg:
Класс: truck, Вероятность: 93.97%
Время обработки этого изображения: 6.95 секунд

Results for /content/test_images/1.jpg:
Класс: airplane, Вероятность: 96.14%
Время обработки этого изображения: 3.01 секунд

Results for /content/test_images/4.jpg:
Класс: cat, Вероятность: 96.81%
Класс: couch, Вероятность: 54.19%
Время обработки этого изображения: 2.98 секунд

Results for /content/test_images/8.jpg:
Класс: horse, Вероятность: 94.93%
Время обработки этого изображения: 4.49 секунд

Results for /content/test_images/5.jpg:
Класс: sheep, Вероятность: 69.74%
Время обработки этого изображения: 3.39 секунд

Results for /content/test_images/3.jpg:
Класс: bird, Вероятность: 96.41%
Время обработки этого изображения: 3.01 секунд

Results for /content/test_images/9.jpg:
Класс: boat, Вероятность: 87.01%
Время обработки этого изображения: 5.75 секунд

Results for /content/test_images/6.jpg:
Класс: dog, Вероятность: 97.93%
В

In [ ]:
import os
import glob
import requests
import time

# Проверяем, что нет запущенных экземпляров uvicorn
os.system('pkill uvicorn')

# Запускаем сервер FastAPI с помощью uvicorn
!nohup uvicorn --port 8090 fastapiapp:app &

# Задержка для запуска сервера
time.sleep(5)

url = 'http://127.0.0.1:8090/predict'
image_folder = '/content/test_images'

# Получаем список всех изображений в папке
image_paths = glob.glob(os.path.join(image_folder, '*'))

# Переменная для сохранения результатов
fastapi_results_summary = []

# Отправляем изображения на сервер по одному
files = [('images', (os.path.basename(image_path), open(image_path, 'rb'), 'image/jpeg')) for image_path in image_paths]
response = requests.post(url, files=files)

if response.status_code == 200:
    data = response.json()
    fastapi_results_summary.append(data)
    for i, res in enumerate(data['results']):
        print(f'Results for image {i+1}:')
        for obj in res['objects']:
            print(f"Класс: {obj['class_name']}, Вероятность: {obj['score']:.2f}%")
        print(f"Время обработки этого изображения: {res['processing_time']:.2f} seconds\n")
else:
    print(f"Error processing images: {response.text}")

# Выводим общую информацию по скорости обработки всех изображений
total_time = data['total_processing_time']
average_time = data['average_processing_time']

print(f"Время обработки всех изображений: {total_time:.2f} сек")
print(f"Среднее время обработки одного изображения: {average_time:.2f} сек")


nohup: appending output to 'nohup.out'
Results for image 1:
Класс: truck, Вероятность: 93.97%
Время обработки этого изображения: 6.25 seconds

Results for image 2:
Класс: airplane, Вероятность: 96.14%
Время обработки этого изображения: 3.00 seconds

Results for image 3:
Класс: cat, Вероятность: 96.81%
Класс: couch, Вероятность: 54.19%
Время обработки этого изображения: 3.70 seconds

Results for image 4:
Класс: horse, Вероятность: 94.93%
Время обработки этого изображения: 3.79 seconds

Results for image 5:
Класс: sheep, Вероятность: 69.74%
Время обработки этого изображения: 2.93 seconds

Results for image 6:
Класс: bird, Вероятность: 96.41%
Время обработки этого изображения: 2.51 seconds

Results for image 7:
Класс: boat, Вероятность: 87.01%
Время обработки этого изображения: 3.66 seconds

Results for image 8:
Класс: dog, Вероятность: 97.93%
Время обработки этого изображения: 4.30 seconds

Results for image 9:
Класс: bird, Вероятность: 80.95%
Время обработки этого изображения: 2.92 seco

# '4. Ответом на ДЗ может быть таблица с результатами тестирования'

In [ ]:
import pandas as pd

comparison_data = {
    'Сервер': ['Gunicorn', 'Uvicorn'],
    'Время затраченное на обработку батча фотографий (s)': [
        total_time_flask,
        total_time
    ]}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df)


     Сервер  Время затраченное на обработку батча фотографий (s)
0  Gunicorn                                          39.371388  
1   Uvicorn                                          36.022598  


# Тесты

In [ ]:
%%file flaskapp1.py
from flask import Flask, request, jsonify
from ultralytics import YOLO

app = Flask(__name__)

model = YOLO("yolov9e.pt")

@app.route('/predict', methods=['POST'])
def predict():
    if 'images' in request.files:
        images = request.files.getlist('images')

        for img in images:
            # Получаем предсказания модели
            model(img.read())

        return jsonify({'message': 'Predictions completed'})

    return jsonify({'error': 'No images provided'}), 400

if __name__ == '__main__':
    app.run(debug=True)


Overwriting flaskapp1.py


In [ ]:
import os
import glob
import requests
import time

# Убеждаемся, что нет запущенных экземпляров gunicorn
os.system('pkill gunicorn')

# Запускаем сервер
!nohup gunicorn --bind 0.0.0.0:8000 flaskapp1:app &

# Задержка для запуска сервера
time.sleep(5)

url = 'http://0.0.0.0:8000/predict'
image_folder = '/content/test_images'

# Получаем список всех изображений в папке
image_paths = glob.glob(os.path.join(image_folder, '*'))

total_start_time = time.time()

# Отправляем изображения на сервер по одному
for image_path in image_paths:
    start_time = time.time()
    with open(image_path, 'rb') as f:
        files = {'images': f}
        response = requests.post(url, files=files)

    if response.status_code == 200:
        print(f"Response for {image_path}: {response.json()}")
    else:
        print(f"Error for {image_path}: {response.text}")

    end_time = time.time()
    print(f"Processing time for {image_path}: {end_time - start_time:.2f} seconds")

total_end_time = time.time()
print(f"Total processing time: {total_end_time - total_start_time:.2f} seconds")


nohup: appending output to 'nohup.out'
Error for /content/test_images/2.jpg: <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

Processing time for /content/test_images/2.jpg: 2.72 seconds
Error for /content/test_images/8.jpg: <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

Processing time for /content/test_images/8.jpg: 0.01 seconds
Error for /content/test_images/7.jpg: <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Eithe

In [ ]:
!pip -q install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.6 MB/s eta 0:00:00


In [ ]:
%%capture
!pip uninstall ultralytics -y

In [ ]:
!pip -q install ultralytics[yolo]


In [ ]:
%%capture
!pip uninstall ultralytics -y
!pip install ultralytics==8.0.14[yolo]

# Данные из учебного ноутбука:

In [ ]:
!pip install gunicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 1.6 MB/s eta 0:00:00


Загрузите модель

In [ ]:
import gdown
# Загрузка из облака yandexcloud
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l6/model_fmr_all.h5', None, quiet=True)

'model_fmr_all.h5'

Загрузите и распакуйте архив с примерами картинок

In [ ]:
import gdown
# Загрузка примеров для распознавания (классификации)
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/knowledge/test_images.zip', None, quiet=True)
# Распаковка архива
!unzip -qo test_images.zip

Создайте файл `flaskapp.py`, запустив ячейку с магической командой: `%%file`

In [ ]:
%%file flaskapp.py
import io
import os
from PIL import Image

import numpy as np
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import img_to_array
import time

app = Flask(__name__)

# Загрузка модели
model = load_model('model_fmr_all.h5')

classes = {0: 'самолет',
            1: 'автомобиль',
            2: 'птица',
            3: 'кот',
            4: 'олень',
            5: 'собака',
            6: 'лягушка',
            7: 'лошадь',
            8: 'корабль',
            9: 'грузовик'}

@app.route('/predict', methods=['POST'])
def predict():
    if request.files.get('image'):
        time.sleep(1)
        image = request.files['image'].read()
        image = Image.open(io.BytesIO(image))
        if image.mode != "RGB":
          image = image.convert("RGB")
        image = image.resize((32, 32))
        image = np.array(image, dtype='float64') / 255
        image = np.expand_dims(image, axis=0)

        preds = model.predict(image)
        class_id = np.argmax(preds)

        return jsonify({'class': classes[int(class_id)]})

    return jsonify({'error': 'No image provided'}), 400

if __name__ == '__main__':
    app.run(debug=True)

Writing flaskapp.py


Если запущен сервер, то закройте его.

In [ ]:
!pkill gunicorn

Запустите сервер

In [ ]:
!nohup gunicorn --bind 0.0.0.0:8000 flaskapp:app &

nohup: appending output to 'nohup.out'


Отправьте тестовую картинку на сервер для распознавания

In [ ]:
url = 'http://0.0.0.0:8000/predict'
image_path = '/content/test_images/2.jpg'

import requests

# Открываем файл "image.jpg" в режиме бинарного чтения
with open(image_path, 'rb') as f:
    # Создаем словарь данных для отправки
    files = {'image': f}

    # Отправляем POST-запрос на сервер
    response = requests.post(url, files=files)

# Проверяем статус ответа
if response.status_code == 200:
    # Распечатываем полученные данные
    data = response.json()
    print('Class ID:', data['class'])
else:
    print(response.text)

Class ID: автомобиль


Установите библиотеки `fastapi python-multipart` и сервер `uvicorn`

In [ ]:
%%capture
!pip install fastapi python-multipart uvicorn

Создайте файл `fastapiapp.py`, запустив ячейку с магической командой: `%%file`

In [ ]:
%%file fastapiapp.py
import io
from PIL import Image

import numpy as np
from fastapi import FastAPI, UploadFile, File
from tensorflow.keras.models import load_model
import time
from starlette.responses import JSONResponse

app = FastAPI()

# Загрузка модели
model = load_model('model_fmr_all.h5')

classes = {0: 'самолет',
            1: 'автомобиль',
            2: 'птица',
            3: 'кот',
            4: 'олень',
            5: 'собака',
            6: 'лягушка',
            7: 'лошадь',
            8: 'корабль',
            9: 'грузовик'}

@app.post('/predict')
async def predict(image: UploadFile = File(...)):
    time.sleep(1)
    image_bytes = await image.read()
    image = Image.open(io.BytesIO(image_bytes))
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize((32, 32))
    image = np.array(image, dtype='float64') / 255
    image = np.expand_dims(image, axis=0)

    preds = model.predict(image)
    class_id = np.argmax(preds)

    response_data = {'class': classes[int(class_id)]}
    return response_data

Writing fastapiapp.py


Если запущен сервер, то закройте его.

In [ ]:
!pkill uvicorn

Запустите сервер

In [ ]:
!nohup uvicorn --port 8090 fastapiapp:app &

nohup: appending output to 'nohup.out'


Проверьте список запущенных процессов

In [ ]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 docker-init
      7 ?        00:00:04 node
     17 ?        00:00:00 oom_monitor.sh
     19 ?        00:00:00 run.sh
     21 ?        00:00:00 kernel_manager_
     23 ?        00:00:00 tail
     59 ?        00:00:08 python3 <defunct>
     60 ?        00:00:01 colab-fileshim.
     82 ?        00:00:04 jupyter-noteboo
     83 ?        00:00:00 dap_multiplexer
    171 ?        00:00:05 python3
    210 ?        00:00:01 python3
    427 ?        00:00:00 gunicorn
    433 ?        00:00:06 gunicorn
   2115 ?        00:00:00 language_servic
   2120 ?        00:00:10 node
   2178 ?        00:00:04 uvicorn
   2201 ?        00:00:00 sleep
   2205 ?        00:00:00 ps


Отправьте тестовую картинку на сервер для распознавания. Предварительно, дождавшись запуска сервера.

In [ ]:
url = 'http://127.0.0.1:8090/predict'
image_path = '/content/test_images/4.jpg'

import requests

# Открываем файл "image.jpg" в режиме бинарного чтения
with open(image_path, 'rb') as f:
    # Создаем словарь данных для отправки
    files = {'image': f}

    # Отправляем POST-запрос на сервер
    response = requests.post(url, files=files)

# Проверяем статус ответа
if response.status_code == 200:
    # Распечатываем полученные данные
    data = response.json()
    print('Class:', data['class'])
else:
  print('Error')

Class: птица
